<img src="NotebookAddons/blackboard-banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>Mapping Areas of Active Agriculture using SAR Time Series Data and Coefficient of Variation</b></font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, <a href="http://earthbigdata.com/" target="_blank">Earth Big Data, LLC</a> </b> <br>
<img style="padding: 7px" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /></font>

<font size="3">This notebook introduces you to a simple method to deliniate areas of active agriculture from SAR time series data. The method is based on the <b>Coefficient of Variation</b> and results in the detection of actively managed crop areas. The coefficient of vatiation approach will be used as the main algorithm for agriculture mapping by the upcoming NASA/ISRO SAR mssion NISAR. Hence, the approach demonstrated here can be applied to data from this future mission.
    
The exercises will use *Jupyter Notebooks*, an environment that is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. <br>

<b>This notebook covers the following data analysis concepts:</b>

- How to calculate the Coefficient of Variation from time series SAR data.
- How to use Coefficient of Variation to indicate areas of active agriculture.
</font>


</font>

<hr>
<font face="Calibri" size="5" color='rgba(200,0,0,0.2)'> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://pandas.pydata.org/" target="_blank">Pandas</a></b> is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. </li>
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
    <li> The <b><a href="https://www.pydoc.io/pypi/asf-hyp3-1.1.1/index.html" target="_blank">asf-hyp3 API</a></b> provides useful functions and scripts for accessing and processing SAR data via the Alaska Satellite Facility's Hybrid Pluggable Processing Pipeline, or HyP3 (pronounced "hype"). </li>
<li><b><a href="https://www.scipy.org/about.html" target="_blank">SciPY</a></b> is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics. </li>

</font>

<font face="Calibri" size="3"> Our first step is to <b>import them:</b> </font>

In [ ]:
%%capture
import os # for chdir, getcwd, path.exists

import pandas as pd # for DatetimeIndex
from osgeo import gdal # for Info
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import rc

from IPython.display import HTML


import asf_notebook as asfn
asfn.jupytertheme_matplotlib_format()


<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Data Stack</b> </font> <img src="NotebookAddons/BogotaAg.jpg" width="350" style="padding:5px;" align="right" /> 

<font size="3"> This notebook will work with a 46-image deep C-band SAR data stack south of Bogota, Colombia, covering the southern end of the city and adjacent agriculture areas. The closeness of the area to Bogota should provide means for visual validation of SAR-derived agriculture extent information. The C-band data were acquired by ESA's Sentinel-1 SAR sensor constellation and are available to you through the services of the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility</a>. 

The site in question is shown in the image to the right. Data were acquired between January of 2017 and December of 2018 in descending orbit direction. 
</font></font>
<br><br>
<font face="Calibri" size="3">Before we get started, let's first <b>create a working directory for this analysis and change into it:</b> </font>

In [ ]:
path = "/home/jovyan/notebooks/SAR_Training/English/Ecosystems/AgricultureMapping"
asfn.new_directory(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

<font face="Calibri" size="3">We will <b>retrieve the relevant data</b> from an <a href="https://aws.amazon.com/" target="_blank">Amazon Web Service (AWS)</a> cloud storage bucket <b>using the following command</b>:</font></font>

In [ ]:
time_series_path = 's3://asf-jupyter-data/BogotaAg.zip'
time_series = os.path.basename(time_series_path)
!aws --region=us-east-1 --no-sign-request s3 cp $time_series_path $time_series

<font face="Calibri" size="3"> Now, let's <b>unzip the file (overwriting previous extractions) and clean up after ourselves:</b> </font>

In [ ]:
if asfn.path_exists(time_series):
    asfn.asf_unzip(os.getcwd(), time_series)
    os.remove(time_series)

<br>
<font face="Calibri" size="5"> <b> 2. Define Some Python Helper Functions for this Notebook </b> </font> 
<br><br>
<font face="Calibri" size="3">We are defining two helper functions for this notebook:

- **create_geotiff()** to write out images
- **timeseries_metrics()** to compute various metrics from a time series data stack</font>

In [ ]:
def create_geotiff(name, array, data_type, ndv, bandnames=None, ref_image=None, 
                  geo_t=None, projection=None):
    # If it's a 2D image we fake a third dimension:
    if len(array.shape) == 2:
        array = np.array([array])
    if ref_image == None and (geo_t == None or projection == None):
        raise RuntimeWarning('ref_image or settings required.')
    if bandnames != None:
        if len(bandnames) != array.shape[0]:
            raise RuntimeError('Need {} bandnames. {} given'
                               .format(array.shape[0],len(bandnames)))
    else:
        bandnames = ['Band {}'.format(i+1) for i in range(array.shape[0])]
    if ref_image != None:
        refimg = gdal.Open(ref_image)
        geo_t = refimg.GetGeoTransform()
        projection = refimg.GetProjection()
    driver = gdal.GetDriverByName('GTIFF')
    array[np.isnan(array)] = ndv
    DataSet = driver.Create(name, array.shape[2], 
                            array.shape[1], array.shape[0], 
                            data_type)
    DataSet.SetGeoTransform(geo_t)
    DataSet.SetProjection(projection)
    for i, image in enumerate(array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
        DataSet.GetRasterBand(i).SetNoDataValue(ndv)
        DataSet.SetDescription(bandnames[i-1])
    DataSet.FlushCache()
    return name

In [ ]:
def timeseries_metrics(raster, ndv=0): 
    # Make us of numpy nan functions
    # Check if type is a float array
    if not raster.dtype.name.find('float')>-1:
        raster=raster.astype(np.float64)
    # Set ndv to nan
    if ndv != np.nan:
        raster[np.equal(raster,ndv)]=np.nan
    # Build dictionary of the metrics
    tsmetrics={}
    rperc = np.nanpercentile(raster,[5,50,95],axis=0)
    tsmetrics['mean']=np.nanmean(raster,axis=0)
    tsmetrics['max']=np.nanmax(raster,axis=0)
    tsmetrics['min']=np.nanmin(raster,axis=0)
    tsmetrics['range']=tsmetrics['max']-tsmetrics['min']
    tsmetrics['median']=rperc[1]
    tsmetrics['p5']=rperc[0]
    tsmetrics['p95']=rperc[2]
    tsmetrics['prange']=rperc[2]-rperc[0]
    tsmetrics['var']=np.nanvar(raster,axis=0)
    tsmetrics['cov']=tsmetrics['var']/tsmetrics['mean']
    return tsmetrics

<br>
<font face="Calibri" size="5"> <b> 3. Define Data Directory and Path to VRT </b> </font> 
<br><br>
<font face="Calibri" size="3"><b>Create a variable containing the VRT filename and the image acquisition dates:</b></font>

In [ ]:
!gdalbuildvrt -separate raster_stack.vrt tiffs/*_VV.tiff
image_file = "raster_stack.vrt"

<font face="Calibri" size="3"><b>Create an index of timedelta64 data with Pandas:</b></font>

In [ ]:
!ls tiffs/*_VV.tiff | sort | cut -c 7-21 > raster_stack.dates
datefile='raster_stack.dates'
dates=open(datefile).readlines()
tindex=pd.DatetimeIndex(dates)

<font face="Calibri" size="3"><b>Print the bands and dates for all images in the virtual raster table (VRT):</b></font>

In [ ]:
j = 1
print(f"Bands and dates for {image_file}")
for i in tindex:
    print("{:4d} {}".format(j, i.date()), end=' ')
    j += 1
    if j%5 == 1:
        print()

<hr>
<br>
<font face="Calibri" size="5"> <b> 4. Create a Time Series Animation to get an Idea of the Dynamics at the Site </b> </font>

<font face="Calibri" size="4"> <b> 4.1 Load Time Series Stack </b> </font>

<font face="Calibri" size="3">Now we are ready to create a time series animation from the calibrated SAR data.
<br><br>
<b>First, create a raster from band 0 and a raster stack from all the images:</b>
</font> 

In [ ]:
img = gdal.Open(image_file)
band = img.GetRasterBand(1)
raster0 = band.ReadAsArray()
band_number = 0 # Needed for updates
rasterstack= img.ReadAsArray()

<font face="Calibri" size="3"><b>Print the bands, pixels, and lines:</b></font>

In [ ]:
print(f"Number of  bands: {img.RasterCount}")
print(f"Number of pixels: {img.RasterXSize}")
print(f"Number of  lines: {img.RasterYSize}")

<br>
<font face="Calibri" size="4"> <b> 4.2 Calibration and Data Conversion between dB and Power Scales </b> </font>

<font face="Calibri" size="3"> <font color='rgba(200,0,0,0.2)'> <b>Note, that if your data were generated by HyP3, this step is not necessary!</b> HyP3 performs the full data calibration and provides you with calibrated data in power scale. </font>
    
If, your data is from a different source, however, calibration may be necessary to ensure that image gray values correspond to proper radar cross section information. 

Calibration coefficients for SAR data are often defined in the decibel (dB) scale due to the high dynamic range of the imaging system. For the L-band ALOS PALSAR data at hand, the conversion from uncalibrated DN values to calibrated radar cross section values in dB scale is performed by applying a standard **calibration factor of -83 dB**. 
<br> <br>
$\gamma^0_{dB} = 20 \cdot log10(DN) -83$

The data at hand are radiometrically terrain corrected images, which are often expressed as terrain flattened $\gamma^0$ backscattering coefficients. For forest and land cover monitoring applications $\gamma^o$ is the preferred metric.

<b>To apply the calibration constant for your data and export in *dB* scale, uncomment the following code cell</b>: </font> 

In [ ]:
 #caldB=20*np.log10(rasterstack)-83

<font face="Calibri" size="3"> While **dB**-scaled images are often "visually pleasing", they are often not a good basis for mathematical operations on data. For instance, when we compute the mean of observations, it makes a difference whether we do that in power or dB scale. Since dB scale is a logarithmic scale, we cannot simply average data in that scale. 
    
Please note that the **correct scale** in which operations need to be performed **is the power scale.** This is critical, e.g. when speckle filters are applied, spatial operations like block averaging are performed, or time series are analyzed.

To **convert from dB to power**, apply: $\gamma^o_{pwr} = 10^{\frac{\gamma^o_{dB}}{10}}$ </font>

In [ ]:
#calPwr=np.power(10.,caldB/10.)

<br>
<font face="Calibri" size="4"> <b> 4.3 Create Time Series Animation </b> </font>

<font face="Calibri" size="3"><b>Create and move into a directory in which to store our plots and animations:</b></font> 

In [ ]:
os.chdir(path)
product_path = 'plots_and_animations'
asfn.new_directory(product_path)
if asfn.path_exists(product_path) and os.getcwd() != f"{path}/{product_path}":
    os.chdir(product_path)
print(f"Current working directory: {os.getcwd()}")

In [ ]:
%%capture 
fig = plt.figure(figsize=(14, 8))
ax = fig.subplots()
ax.axis('off')
vmin = np.percentile(rasterstack.flatten(), 5)
vmax = np.percentile(rasterstack.flatten(), 95)

r0dB = 20 * np.log10(raster0) - 83

im = ax.imshow(raster0, cmap='gray', vmin=vmin, vmax=vmax)
ax.set_title("{}".format(tindex[0].date()))

def animate(i):
    ax.set_title("{}".format(tindex[i].date()))
    im.set_data(rasterstack[i])

# Interval is given in milliseconds
ani = animation.FuncAnimation(fig, animate, frames=rasterstack.shape[0], interval=400)

<font face="Calibri" size="3"><b>Configure matplotlib's RC settings for the animation:</b></font> 

In [ ]:
rc('animation', embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation

<font face="Calibri" size="3"><b>Create a javascript animation of the time-series running inline in the notebook:</b></font> 

In [ ]:
HTML(ani.to_jshtml())

<font face="Calibri" size="3"><b>Delete the dummy png</b> that was saved to the current working directory while generating the javascript animation in the last code cell.</font> 

In [ ]:
try:
    os.remove('None0000000.png')
except FileNotFoundError:
    pass

<font face="Calibri" size="3"><b>Save the animation (animation.gif):</b> </font> 

In [ ]:
ani.save('animation-AgSite.gif', writer='pillow', fps=2)

<hr>
<br>
<font face="Calibri" size="5"> <b> 5. Computation and Visualization of Time Series Metrics</b> </font>

<font face="Calibri" size="3">Once a time-series was constructed, we can compute <b>a set of metrics</b> that will aid us later in applications such as <i>change detection and active agriculture detection</i>. In the next code cells, we will compute the following variables for each pixel in the stack:

- Mean 
- Median
- Maximum
- Minimum
- Range (Maximum - Minimum)
- 5th Percentile
- 95th Percentile
- PRange (95th - 5th Percentile)
- Variance
- Coefficient of Variation (Variance/Mean)

<hr>
First, we <b>mask out pixels</b> without relevant information to be unbiased in statical number we calculate later. Then we <b>calculate the time series metrics</b>:
</font> 

In [ ]:
mask = rasterstack == 0
rasterPwr = np.ma.array(rasterstack, mask=mask, dtype=np.float64)

In [ ]:
%%capture
metrics = timeseries_metrics(rasterPwr.filled(np.nan), ndv=np.nan)

In [ ]:
metrics.keys()

<font face="Calibri" size="3">Let's look at the histograms for the time series variance and coeficient of variation to aid displaying those images:</font> 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,4))
ax[0].hist(metrics['var'].flatten(), bins=100, range=(0, 0.005))
ax[1].hist(metrics['cov'].flatten(), bins=100, range=(0, 0.04))
_ = ax[0].set_title('Variance')
_ = ax[1].set_title('Coefficient of Variation')

<font face="Calibri" size="3">We use thresholds determined from those histograms to set the scaling in the time series visualization. For the backscatter metrics we choose a typical range appropriate for this ecosystem, the C-band radar sensor, and the VH polarization. A typical range is -30 dB (0.0001) to -10 dB (0.1).</font> 

In [ ]:
# List the metrics keys you want to plot
metric_keys = ['mean', 'median', 'max',
               'min', 'p95', 'p5', 'range', 
               'prange', 'var', 'cov']
fig = plt.figure(figsize=(16, 40))
idx = 1
for i in metric_keys:
    ax = fig.add_subplot(5,2,idx)
    if i == 'var': vmin, vmax = (0.0, 0.003)
    elif i == 'cov': vmin, vmax = (0., 0.03)
    else:
        vmin,vmax=(0.0,0.3)
    ax.imshow(metrics[i],vmin=vmin,vmax=vmax,cmap='gray')
    ax.set_title(i.upper())
    ax.axis('off')
    idx+=1

<hr>
<br>
<font face="Calibri" size="5"> <b> 6. Use Coefficient of Variation for Mapping Active Agriculture</b> </font>

<font face="Calibri" size="3">Areas of active agriculture show strong changes of vegetation throughout a growth season. From plantation through maturation and harvest, the strong variation of vegetation structure causes the radar brigthness to vary significantly throughout a season, resulting in large values for <i>Range</i>, <i>Variance</i>, and <i>Coefficient of Variation</i> metrics. Steps for agriculture mapping using Coefficient of Variation include:

- RTC process your data (done outside of notebook)
- Stack up your data (beginning of notebook)
- Calculate the Coefficient of Variation (CoV) metrics
- Threshold the Coefficient of Variation to create candidate areas for active agriculture
- Export map as a GeoTIFF for analysis in a GIS.
</font> 

With the CoV metric already calculated above, we can now <b>move toward thresholding the metric</b>:
</font> 

<br>
<font face="Calibri" size="3">We can set a threshold $t$ for the <b>coefficient of variation image</b>
to classify change in the time series:
    
${cp}_{x,y} = \frac{\sigma_{x,y}^2}{\overline{X}_{x,y}} > t$ 

Let's look at the histogram and the Cumulative Distribution Function (CDF) of the coefficient of variation:</font> 

In [ ]:
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(14, 4)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(122)

h = ax1.hist(metrics['cov'].flatten(), bins=200, range=(0, 0.03))
ax1.xaxis.set_label_text('Coefficient of Variation Metric')
ax1.set_title('Histogram')

n, bins, patches = ax2.hist(metrics['cov'].flatten(), bins=200, 
                            range=(0, 0.03), cumulative='True', 
                            density='True', histtype='step', 
                            label='Empirical')
ax2.xaxis.set_label_text('Coefficient of Variation Metric')
ax2.set_title('CDF')
plt.savefig('thresh_cov_histogram.png', dpi=200, transparent='true')

outind = np.where(n > 0.9)
threshind = np.min(outind)
threshcov = bins[threshind]
ax1.axvline(threshcov,color='red')
ax2.axvline(threshcov,color='red')

<br>
<font face="Calibri" size="3">With a threshold of $t=CDF_{cov^2} > 0.90$ (10% pixels with highest variance) the change pixels would look like the following image:</font> 

In [ ]:
plt.figure(figsize=(8, 8))
mask = metrics['cov'] < threshcov # For display we prepare the inverse mask
maskcov = ~mask # Store this for later output
plt.imshow(mask, cmap='gray')
_ = plt.title('Threshold Classifier on Coefficient of Variation > %1.3f' % threshcov )
plt.savefig('changes_cov_threshold.png', dpi=200, transparent='true')

<hr>
<br>
<font face="Calibri" size="5"> <b> 7. Write Our Change Detection Results and Metrics Images to GeoTIFF files</b> </font>

<font face="Calibri" size="4"> <b> 7.1 Determine Output Geometry </b> </font>

<font face="Calibri" size="3">First, we need to <b>set the correct geotransformation and projection information</b>. We retrieve the values from the input images:
</font> 

In [ ]:
proj = img.GetProjection()
geotrans = list(img.GetGeoTransform())
geotrans

<br>
<font face="Calibri" size="4"> <b> 7.2 Output Time Series Metrics Images </b> </font>

<font face="Calibri" size="3">We use the root of the time series data stack name and append a _ts_metrics_&lt;metric&gt;.tif ending as filenames:
</font> 

In [ ]:
# Time Series Metrics as image image:
# We make a new subdirectory where we will store the images
dirname=image_file.replace('.vrt','_tsmetrics')
os.makedirs(dirname,exist_ok=True)
print(dirname)

<br>
<font face="Calibri" size="3">Now we can <b>output the individual metrics as GeoTIFF images</b>:
</font> 

In [ ]:
names = [] # List to keep track of all the names
for i in metrics:
    # Name, Array, DataType, NDV,bandnames=None,ref_image
    name = os.path.join(dirname,image_file.replace('.vrt','_'+i+'.tif'))
    create_geotiff(name, metrics[i], gdal.GDT_Float32,np.nan, 
                   [i], geo_t=geotrans, projection=proj)
    names.append(name)

<br>
<font face="Calibri" size="4"> <b> 7.3 Output Detected Agriculture Areas Map </b> </font>

<font face="Calibri" size="3">Now we can <b>output the map of active agriculture that we have created using the Coefficient of Variations metric. We will output to a GeoTIFF image</b>:
</font> 

In [ ]:
imagenamecov = image_file.replace('.vrt', 'Ag_from_cov_threshold.tif')
create_geotiff(imagenamecov, maskcov, gdal.GDT_Byte, 
               np.nan, [i], geo_t=geotrans, projection=proj)

<hr>
<br>
<font face="Calibri" size="5"> <b> 8. Conclusion</b> </font>

<font face="Calibri" size="3">The Coefficient of Variation is a good indicator for agricultural activity, however, by itself it is not fail safe. Further analysis of radar brightness can be performed. Also, other remote sensing data can be added to remove false alarms. 

For a bit more information on change detection and SAR in general, please look at the recently published <a href="https://gis1.servirglobal.net/TrainingMaterials/SAR/SARHB_FullRes.pdf" target="_blank">SAR Handbook: Comprehensive Methodologies for Forest Monitoring and Biomass Estimation</a>.
</font> 

<hr>
<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISES</u> </font></b> 

<font face="Calibri" size="3"> Explore agriculture mapping from the 46-image deep data stack a bit more by:

<ul>
  <li>Change the threshold used for identifying active agriculture areas.</li>
  <li>Load created agriculture extent maps into QGIS and compare the detected areas with other image data (e.g., ESRI satellite layers). What do you think about the quality of the created information? What are the main error sources that you can see? What would be possible methods to mitigate these errors?</li>
</ul>

</font>
</div>
<br>
<hr>

<font face="Calibri" size="2"> <i>Exercise8-AgricultureMappingwithSARUsingCoV.ipynb - Version 1.3.0 - April 2021
    <br>
        <b>Version Changes:</b>
    <ul>
        <li>from osgeo import gdal</li>
        <li>namespace asf_notebook</li>
    </ul>
    </i>
</font>